In [1]:
import pandas as pd
from pathlib import Path

In [2]:
BASE_DATA_DIR = Path("data/")
HTML_BASE_DIR = BASE_DATA_DIR / 'html'
assert HTML_BASE_DIR.exists()

In [3]:
!ls data/1973_10.html

ls: data/1973_10.html: No such file or directory


Verify they're all parseable:

In [4]:
EXPECTED_COLUMNS = ['T Media', 'T min', 'T max', 'Precip.', 'Umidità', 'Vento Max',]

In [16]:
issues = set()
successful_dfs = {}
for html_path in HTML_BASE_DIR.glob("*.html"):
    year, month = html_path.name.replace('.html', '').split("_")
    year, month = (int(year), int(month))
    try:
        with html_path.open() as fp:
            dirty_df = pd.read_html(fp.read(), attrs = {'id': 'table-meteo-archivio'})[0]
            assert (set(EXPECTED_COLUMNS) - set(dirty_df.columns)) == set()
            dirty_df['Timestamp'] = dirty_df['Giorno'].apply(lambda g: pd.Timestamp(year, month, g))
            dirty_df.set_index('Timestamp', inplace=True)
        successful_dfs[(year, month)] = dirty_df
    except Exception as exc:
        issues.add(html_path.name)
        raise exc
print("All good!")

All good!


In [18]:
import itertools
def parse_string_column(column, *remove_sequences, known_sequences=['-', 'n/d']):
    column = column.copy()
    for seq in itertools.chain(remove_sequences, known_sequences):
        column = column.str.replace(seq, '')
    return pd.to_numeric(column)

In [26]:
clean_dataframes = []
for (year, month), dirty_df in successful_dfs.items():
    try:
        clean_df = pd.concat([
            
            parse_string_column(dirty_df['T Media'], '°C').rename('Mean Temperature'),
            parse_string_column(dirty_df['T min'], '°C').rename('Min Temperature'),
            parse_string_column(dirty_df['T max'], '°C').rename('Max Temperature'),

            parse_string_column(dirty_df['Vento Max'], 'km/h').rename('Max Wind'),
            parse_string_column(dirty_df['Umidità'], '%').rename('Humidity') / 100,

            parse_string_column(dirty_df['Precip.'], 'mm').rename('Rainfall'),

        ], axis=1)
    except Exception as exc:
        print(f"Issues with {year}, {month}")
        raise exc
    clean_dataframes.append(clean_df)

In [31]:
df = pd.concat(clean_dataframes).sort_index()

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18494 entries, 1973-01-01 to 2023-08-20
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Mean Temperature  18438 non-null  float64
 1   Min Temperature   18436 non-null  float64
 2   Max Temperature   18436 non-null  float64
 3   Max Wind          16792 non-null  float64
 4   Humidity          18375 non-null  float64
 5   Rainfall          3220 non-null   float64
dtypes: float64(6)
memory usage: 1011.4 KB


In [37]:
df.head()

,Mean Temperature,Min Temperature,Max Temperature,Max Wind,Humidity,Rainfall
Timestamp,,,,,,
1973-01-01,4.3,1.0,6.0,20.2,0.95,42.0
1973-01-02,4.3,2.0,6.0,9.1,0.97,15.0
1973-01-03,5.9,3.0,10.0,7.2,0.94,6.0
1973-01-04,3.7,1.0,10.0,5.4,0.92,NaN
1973-01-05,4.5,1.0,7.0,5.4,0.90,NaN


In [38]:
df.to_csv('data/torino_meteo.csv')

In [39]:
!head data/torino_meteo.csv

Timestamp,Mean Temperature,Min Temperature,Max Temperature,Max Wind,Humidity,Rainfall
1973-01-01,4.3,1.0,6.0,20.2,0.95,42.0
1973-01-02,4.3,2.0,6.0,9.1,0.97,15.0
1973-01-03,5.9,3.0,10.0,7.2,0.94,6.0
1973-01-04,3.7,1.0,10.0,5.4,0.92,
1973-01-05,4.5,1.0,7.0,5.4,0.9,
1973-01-06,0.6,2.0,3.0,,0.93,
1973-01-07,0.1,3.0,5.0,,0.84,
1973-01-08,1.9,3.0,10.0,,0.74,
1973-01-09,1.6,3.0,8.0,,0.76,
